#   LangChain 활용 에이전트 활용하기 (ReAct) + 라우팅 기반 다국어 RAG 구현하기

### **학습 목표:** ReAct 프레임워크를 활용한 에이전트 시스템을 개발한다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import warnings
warnings.filterwarnings("ignore")

---

##  **다국어 RAG 시스템 (LangChain 구현)**

- 옵션 1: **다국어 RAG 시스템**은 언어 교차 검색 기능을 통해 다양한 언어의 문서를 처리함

- 옵션 2: **언어 감지**와 **자동번역** 기능이 통합되어 seamless한 다국어 처리가 가능함

- 옵션 3: **벡터저장소 라우팅**을 통해 각 언어별 최적화된 처리 경로를 구성할 수 있음


---

### 1. **언어 교차(cross-lingual) 검색** 

- **언어 교차 검색**은 서로 다른 언어 간의 정보 검색을 가능하게 하는 기술임

- 질의어와 문서가 **다른 언어**여도 의미적 연관성을 기반으로 검색이 가능함

- **다국어 임베딩**을 활용하여 언어 간 의미적 매칭을 수행함

`(1) 다국어 문서 로드 및 전처리` 

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 현재 작업 디렉토리 확인
print(f"현재 작업 디렉토리: {os.getcwd()}")

# data 폴더 경로 확인
data_dir = os.path.join(os.getcwd(), 'data')
print(f"Data 폴더 경로: {data_dir}")
print(f"Data 폴더 존재 여부: {os.path.exists(data_dir)}")

# data 폴더 내용 확인
if os.path.exists(data_dir):
    print(f"\nData 폴더 내용:")
    for file in os.listdir(data_dir):
        print(f"  - {file}")

# 데이터 로드
def load_text_files(txt_files):
    data = []
    
    print(f"\n로드할 파일 수: {len(txt_files)}")
    for text_file in txt_files:
        print(f"  - 로딩 중: {text_file}")
        try:
            loader = TextLoader(text_file, encoding='utf-8')
            loaded = loader.load()
            print(f"    ✅ 로드된 문서 수: {len(loaded)}, 길이: {len(loaded[0].page_content) if loaded else 0}")
            data += loaded
        except Exception as e:
            print(f"    ❌ 로드 실패: {str(e)[:100]}")
    
    return data

# 한국어 데이터 로드
korean_txt_files = glob(os.path.join(data_dir, '*_KR.md'))
print(f"\n찾은 한국어 파일: {korean_txt_files}")

korean_data = load_text_files(korean_txt_files)
print(f"\n총 로드된 한국어 문서 수: {len(korean_data)}")

if len(korean_data) > 0:
    # 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",    # TikToken 인코더 이름
        separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
        chunk_size=300,            # 문서 분할 크기
        chunk_overlap=50,          # 문서 분할 중첩  
        is_separator_regex=True,      # 구분자가 정규식인지 여부
        keep_separator=True,          # 구분자 유지 여부
    )

    korean_docs = text_splitter.split_documents(korean_data)
    print(f"✅ 한국어 청크 수: {len(korean_docs)}")
else:
    korean_docs = []
    print("⚠️ 한국어 문서가 로드되지 않았습니다!")

현재 작업 디렉토리: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025
Data 폴더 경로: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025\data
Data 폴더 존재 여부: True

Data 폴더 내용:
  - Rivian_EN.md
  - Rivian_KR.md
  - Tesla_EN.md
  - Tesla_KR.md

찾은 한국어 파일: ['f:\\대학과제\\25-하반기\\8_모연_재직자_LLM\\PRJ_03\\Week1_20251021_20251025\\data\\Rivian_KR.md', 'f:\\대학과제\\25-하반기\\8_모연_재직자_LLM\\PRJ_03\\Week1_20251021_20251025\\data\\Tesla_KR.md']

로드할 파일 수: 2
  - 로딩 중: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025\data\Rivian_KR.md
    ✅ 로드된 문서 수: 1, 길이: 786
  - 로딩 중: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025\data\Tesla_KR.md
    ✅ 로드된 문서 수: 1, 길이: 744

총 로드된 한국어 문서 수: 2
✅ 한국어 청크 수: 5


In [4]:
# 영어 데이터 로드
english_txt_files = glob(os.path.join(data_dir, '*_EN.md'))
print(f"찾은 영어 파일: {english_txt_files}")

english_data = load_text_files(english_txt_files)
print(f"\n총 로드된 영어 문서 수: {len(english_data)}")

if len(english_data) > 0:
    # 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name="cl100k_base",    # TikToken 인코더 이름
        separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
        chunk_size=300,            # 문서 분할 크기
        chunk_overlap=50,          # 문서 분할 중첩  
        is_separator_regex=True,      # 구분자가 정규식인지 여부
        keep_separator=True,          # 구분자 유지 여부
    )

    english_docs = text_splitter.split_documents(english_data)
    print(f"✅ 영어 청크 수: {len(english_docs)}")
else:
    english_docs = []
    print("⚠️ 영어 문서가 로드되지 않았습니다!")

찾은 영어 파일: ['f:\\대학과제\\25-하반기\\8_모연_재직자_LLM\\PRJ_03\\Week1_20251021_20251025\\data\\Rivian_EN.md', 'f:\\대학과제\\25-하반기\\8_모연_재직자_LLM\\PRJ_03\\Week1_20251021_20251025\\data\\Tesla_EN.md']

로드할 파일 수: 2
  - 로딩 중: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025\data\Rivian_EN.md
    ✅ 로드된 문서 수: 1, 길이: 1325
  - 로딩 중: f:\대학과제\25-하반기\8_모연_재직자_LLM\PRJ_03\Week1_20251021_20251025\data\Tesla_EN.md
    ✅ 로드된 문서 수: 1, 길이: 1200

총 로드된 영어 문서 수: 2
✅ 영어 청크 수: 2


`(2) 문서 임베딩 및 벡터저장소에 저장`  

- **3가지 임베딩 모델**(OpenAI, HuggingFace, Ollama)을 활용하여 문서 벡터화 및 성능 비교 
- 한국어 지원: **OpenAI**의 text-embedding-3-small, **HuggingFace**의 bge-m3 모델
- 한국어 미지원: **Ollama**의 nomic-embed-text 모델

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_ollama import OllamaEmbeddings

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")
print("✅ OpenAI 임베딩 모델 생성 완료")

# Hugging Face 임베딩 모델 생성
try:
    embeddings_huggingface = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={'device': 'cpu'},  # CPU 사용
        encode_kwargs={'normalize_embeddings': True}  # 정규화 활성화
    )
    print("✅ Hugging Face 임베딩 모델 생성 완료")
except Exception as e:
    print(f"⚠️  Hugging Face 임베딩 모델 생성 실패: {str(e)[:100]}")
    embeddings_huggingface = None

# Ollama 임베딩 모델 생성
try:
    embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text")
    print("✅ Ollama 임베딩 모델 생성 완료")
except Exception as e:
    print(f"⚠️  Ollama 임베딩 모델 생성 실패: {str(e)[:100]}")
    embeddings_ollama = None

✅ OpenAI 임베딩 모델 생성 완료
✅ Hugging Face 임베딩 모델 생성 완료
✅ Hugging Face 임베딩 모델 생성 완료
✅ Ollama 임베딩 모델 생성 완료
✅ Ollama 임베딩 모델 생성 완료


In [6]:
# 다국어 벡터 저장소 구축
from langchain_chroma import Chroma

# 문서 로드 확인
print(f"한국어 문서 수: {len(korean_docs)}")
print(f"영어 문서 수: {len(english_docs)}")
print(f"총 문서 수: {len(korean_docs + english_docs)}")

# 문서가 없으면 에러 메시지 출력
if len(korean_docs) == 0 or len(english_docs) == 0:
    print("\n⚠️ 오류: 문서가 로드되지 않았습니다!")
    print("data 폴더에 *_KR.md 및 *_EN.md 파일이 있는지 확인하세요.")
    print("이전 셀(Cell 9, 10)을 다시 실행하여 문서를 로드하세요.\n")
    db_openai = None
    db_huggingface = None
    db_ollama = None
else:
    # OpenAI 벡터 저장소 생성
    try:
        db_openai = Chroma.from_documents(
            documents=korean_docs+english_docs, 
            embedding=embeddings_openai,
            collection_name="db_openai",
            persist_directory="./chroma_db",
        )
        print(f"✅ OpenAI: {db_openai._collection.count()} 문서")
    except Exception as e:
        print(f"❌ OpenAI: 벡터 저장소 생성 실패 - {str(e)[:150]}")
        db_openai = None

    # Hugging Face 벡터 저장소 생성 (임베딩 모델이 생성된 경우만)
    if embeddings_huggingface is not None:
        try:
            db_huggingface = Chroma.from_documents(
                documents=korean_docs+english_docs, 
                embedding=embeddings_huggingface,
                collection_name="db_huggingface",
                persist_directory="./chroma_db",
            )
            print(f"✅ Hugging Face: {db_huggingface._collection.count()} 문서")
        except Exception as e:
            print(f"❌ Hugging Face: 벡터 저장소 생성 실패 - {str(e)[:100]}")
            db_huggingface = None
    else:
        print("⚠️  Hugging Face: 임베딩 모델이 없어 벡터 저장소 생성 스킵")
        db_huggingface = None

    # Ollama 벡터 저장소 생성 (임베딩 모델이 생성된 경우만)
    if embeddings_ollama is not None:
        try:
            db_ollama = Chroma.from_documents(
                documents=korean_docs+english_docs, 
                embedding=embeddings_ollama,
                collection_name="db_ollama",
                persist_directory="./chroma_db",
            )
            print(f"✅ Ollama: {db_ollama._collection.count()} 문서")
        except Exception as e:
            print(f"❌ Ollama: 벡터 저장소 생성 실패 (Ollama 서버가 실행되지 않았을 수 있습니다)")
            db_ollama = None
    else:
        print("⚠️  Ollama: 임베딩 모델이 없어 벡터 저장소 생성 스킵")
        db_ollama = None

한국어 문서 수: 5
영어 문서 수: 2
총 문서 수: 7
✅ OpenAI: 42 문서
✅ OpenAI: 42 문서
✅ Hugging Face: 42 문서
✅ Hugging Face: 42 문서
❌ Ollama: 벡터 저장소 생성 실패 (Ollama 서버가 실행되지 않았을 수 있습니다)
❌ Ollama: 벡터 저장소 생성 실패 (Ollama 서버가 실행되지 않았을 수 있습니다)


In [7]:
# 다국어 벡터 저장소 로드
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(model="text-embedding-3-small")

# Hugging Face 임베딩 모델 생성
try:
    embeddings_huggingface = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )
except:
    embeddings_huggingface = None

# Ollama 임베딩 모델 생성
try:
    embeddings_ollama = OllamaEmbeddings(model="nomic-embed-text")
except:
    embeddings_ollama = None

# OpenAI 벡터 저장소 로드
db_openai = Chroma(
    embedding_function=embeddings_openai,
    collection_name="db_openai",
    persist_directory="./chroma_db",
)
print(f"OpenAI: {db_openai._collection.count()}")

# Hugging Face 벡터 저장소 로드 (임베딩 모델이 생성된 경우만)
if embeddings_huggingface is not None:
    try:
        db_huggingface = Chroma(
            embedding_function=embeddings_huggingface,
            collection_name="db_huggingface",
            persist_directory="./chroma_db",
        )
        print(f"Hugging Face: {db_huggingface._collection.count()}")
    except Exception as e:
        print(f"Hugging Face: 벡터 저장소 로드 실패")
        db_huggingface = None
else:
    print("Hugging Face: 임베딩 모델이 없어 벡터 저장소 로드 스킵")
    db_huggingface = None

# Ollama 벡터 저장소 로드 (임베딩 모델이 생성된 경우만)
if embeddings_ollama is not None:
    try:
        db_ollama = Chroma(
            embedding_function=embeddings_ollama,
            collection_name="db_ollama",
            persist_directory="./chroma_db",
        )
        print(f"Ollama: {db_ollama._collection.count()}")
    except Exception as e:
        print(f"Ollama: 벡터 저장소 로드 실패")
        db_ollama = None
else:
    print("Ollama: 임베딩 모델이 없어 벡터 저장소 로드 스킵")
    db_ollama = None

OpenAI: 42
Hugging Face: 42
Ollama: 7


`(3) RAG 성능 비교` 

In [8]:
# RAG 체인 생성
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chat_models import init_chat_model

# 질문 템플릿 정의 
template = """Answer the question based only on the following context.
Do not use any external information or knowledge.
If the answer is not in the context, answer "I don't know".

When answering:
- For proper nouns (names of people, places, organizations, etc.), provide both Korean and English names in the format: 한글명(English Name) or English Name(한글명)
- Example: 세종대왕(King Sejong), Microsoft(마이크로소프트), New York(뉴욕)

[Context]
{context}

[Question]
{question}

[Answer]
"""

# 프롬프트 생성
prompt = ChatPromptTemplate.from_template(template)

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# LLM 모델 생성
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 체인 생성
def create_rag_chain(vectorstore):

    retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

    return (
        {"context": retriever | format_docs , "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# 벡터 저장소 확인 및 체인 생성
print("="*80)
print("RAG 체인 생성 중...")
print("="*80)

# OpenAI 체인 생성
if db_openai is not None:
    try:
        rag_chain_openai = create_rag_chain(db_openai)
        print("✅ OpenAI RAG 체인 생성 완료")
    except Exception as e:
        print(f"❌ OpenAI RAG 체인 생성 실패: {str(e)[:100]}")
        rag_chain_openai = None
else:
    print("⚠️  OpenAI: 벡터 저장소가 없어 RAG 체인 생성 스킵")
    print("    → Cell 9, 10을 실행하여 문서를 로드한 후 Cell 13을 다시 실행하세요.")
    rag_chain_openai = None

# Hugging Face 체인 생성 (db_huggingface가 있는 경우만)
if db_huggingface is not None:
    try:
        rag_chain_huggingface = create_rag_chain(db_huggingface)
        print("✅ HuggingFace RAG 체인 생성 완료")
    except Exception as e:
        print(f"❌ HuggingFace RAG 체인 생성 실패: {str(e)[:100]}")
        rag_chain_huggingface = None
else:
    rag_chain_huggingface = None
    print("⚠️  HuggingFace RAG 체인 생성 스킵")

# Ollama 체인 생성 (db_ollama가 있는 경우만)
if db_ollama is not None:
    try:
        rag_chain_ollama = create_rag_chain(db_ollama)
        print("✅ Ollama RAG 체인 생성 완료")
    except Exception as e:
        print(f"❌ Ollama RAG 체인 생성 실패: {str(e)[:100]}")
        rag_chain_ollama = None
else:
    rag_chain_ollama = None
    print("⚠️  Ollama RAG 체인 생성 스킵")

print("="*80)

RAG 체인 생성 중...
✅ OpenAI RAG 체인 생성 완료
✅ HuggingFace RAG 체인 생성 완료
✅ Ollama RAG 체인 생성 완료


In [9]:
# 한국어 쿼리에 대한 성능 평가
query_ko = "테슬라 창업자는 누구인가요?"

print("="*80)
print(f"질문: {query_ko}")
print("="*80)

# OpenAI
if rag_chain_openai is not None:
    try:
        output_openai = rag_chain_openai.invoke(query_ko)
        print("✅ OpenAI:", output_openai)
    except Exception as e:
        print(f"❌ OpenAI 오류: {str(e)[:150]}")
else:
    print("⚠️  OpenAI: 사용 불가")
    print("   → 문서를 로드하지 않았거나 벡터 저장소 생성에 실패했습니다.")
    print("   → Cell 9, 10을 실행하여 문서를 로드한 후 Cell 13, 16을 다시 실행하세요.")

print()

# Hugging Face (사용 가능한 경우만)
if rag_chain_huggingface is not None:
    try:
        output_huggingface = rag_chain_huggingface.invoke(query_ko)
        print("✅ Hugging Face:", output_huggingface)
    except Exception as e:
        print(f"❌ Hugging Face 오류: {str(e)[:150]}")
else:
    print("⚠️  Hugging Face: 사용 불가 (벡터 저장소 생성 실패 또는 임베딩 모델 없음)")

print()

# Ollama (사용 가능한 경우만)
if rag_chain_ollama is not None:
    try:
        output_ollama = rag_chain_ollama.invoke(query_ko)
        print("✅ Ollama:", output_ollama)
    except Exception as e:
        print(f"❌ Ollama 오류: {str(e)[:150]}")
else:
    print("⚠️  Ollama: 사용 불가 (Ollama 서버가 실행되지 않음)")

print("="*80)

질문: 테슬라 창업자는 누구인가요?
✅ OpenAI: 테슬라(Tesla, Inc.)의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.

✅ OpenAI: 테슬라(Tesla, Inc.)의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.

✅ Hugging Face: 테슬라(Tesla, Inc.)의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.

✅ Hugging Face: 테슬라(Tesla, Inc.)의 창업자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)입니다.

❌ Ollama 오류: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
❌ Ollama 오류: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


In [10]:
# 영어 쿼리에 대한 성능 평가
query_en = "Who is the founder of Tesla?"

print("="*80)
print(f"Question: {query_en}")
print("="*80)

# OpenAI
if rag_chain_openai is not None:
    try:
        output_openai = rag_chain_openai.invoke(query_en)
        print("✅ OpenAI:", output_openai)
    except Exception as e:
        print(f"❌ OpenAI 오류: {str(e)[:150]}")
else:
    print("⚠️  OpenAI: 사용 불가")
    print("   → 문서를 로드하지 않았거나 벡터 저장소 생성에 실패했습니다.")
    print("   → Cell 9, 10을 실행하여 문서를 로드한 후 Cell 13, 16을 다시 실행하세요.")

print()

# Hugging Face (사용 가능한 경우만)
if rag_chain_huggingface is not None:
    try:
        output_huggingface = rag_chain_huggingface.invoke(query_en)
        print("✅ Hugging Face:", output_huggingface)
    except Exception as e:
        print(f"❌ Hugging Face 오류: {str(e)[:150]}")
else:
    print("⚠️  Hugging Face: 사용 불가 (벡터 저장소 생성 실패 또는 임베딩 모델 없음)")

print()

# Ollama (사용 가능한 경우만)
if rag_chain_ollama is not None:
    try:
        output_ollama = rag_chain_ollama.invoke(query_en)
        print("✅ Ollama:", output_ollama)
    except Exception as e:
        print(f"❌ Ollama 오류: {str(e)[:150]}")
else:
    print("⚠️  Ollama: 사용 불가 (Ollama 서버가 실행되지 않음)")

print("="*80)

Question: Who is the founder of Tesla?
✅ OpenAI: Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.

✅ OpenAI: Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.

✅ Hugging Face: Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.

✅ Hugging Face: Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.

❌ Ollama 오류: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download
❌ Ollama 오류: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download


---

### 2. **언어 감지 및 번역 자동화** 

- **langdetect**의 **언어 감지** 기능으로 입력 텍스트의 언어를 자동으로 식별함

- **LibreTranslate**를 통해 감지된 언어로 번역 

- LibreTranslate는 오픈소스 번역 API로, 로컬 서버로 실행 가능

- langdetect 설치: pip install langdetect 또는 uv add langdetect

- LibreTranslate 서버 실행:
  ```bash
  H:\miniconda3\envs\traslate_svr\Scripts\libretranslate.exe --host 0.0.0.0 --port 5000 --load-only ko,en,ja
  ```

`(1) 한국어 문서 벡터저장소 초기화 ` 

In [11]:
# 한국어 문서로 저장되어 있는 벡터 저장소 로드

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 39


In [12]:
load_dotenv()

True

In [13]:
import requests
from langdetect import detect

# LibreTranslate 서버 설정
LIBRETRANSLATE_URL = "http://localhost:5000"

# LibreTranslate 번역 클래스
class LibreTranslateClient:
    """LibreTranslate 서버와 통신하는 클라이언트"""
    
    def __init__(self, base_url="http://localhost:5000"):
        self.base_url = base_url
        self.supported_languages = []
        self._check_server()
    
    def _check_server(self):
        """서버 연결 확인 및 지원 언어 목록 저장"""
        try:
            response = requests.get(f"{self.base_url}/languages", timeout=5)
            if response.status_code == 200:
                print(f"✅ LibreTranslate 서버 연결 성공: {self.base_url}")
                languages = response.json()
                self.supported_languages = [lang['code'] for lang in languages]
                
                # 중국어 변형(zh-Hans, zh-Hant)을 zh로 매핑하여 추가
                if 'zh-Hans' in self.supported_languages or 'zh-Hant' in self.supported_languages:
                    if 'zh' not in self.supported_languages:
                        self.supported_languages.append('zh')
                
                print(f"   지원 언어: {self.supported_languages}")
                return True
        except requests.exceptions.RequestException as e:
            print(f"❌ LibreTranslate 서버 연결 실패: {self.base_url}")
            print(f"   오류: {str(e)}")
            print(f"\n서버를 다음 명령어로 실행하세요:")
            print(f"H:\\miniconda3\\envs\\traslate_svr\\Scripts\\libretranslate.exe --host 0.0.0.0 --port 5000 --load-only ko,en,ja")
            return False
    
    def translate_text(self, text, target_lang, source_lang="auto"):
        """
        텍스트 번역
        
        Args:
            text: 번역할 텍스트
            target_lang: 목표 언어 (ko, en, ja, zh 등)
            source_lang: 소스 언어 (기본값: auto)
        
        Returns:
            번역된 텍스트 문자열
        """
        # 언어 코드 정규화 (DeepL 형식 → LibreTranslate 형식)
        target_lang = target_lang.lower().replace('-us', '').replace('-', '')
        if source_lang != "auto":
            source_lang = source_lang.lower().replace('-us', '').replace('-', '')
        
        # zh를 zh-Hans로 변환 (서버에서 지원하는 형식)
        if target_lang == 'zh':
            target_lang = 'zh-Hans'
        if source_lang == 'zh':
            source_lang = 'zh-Hans'
        
        # 언어 지원 확인 (원본 supported_languages 사용)
        server_supported = [lang for lang in self.supported_languages if lang not in ['zh']]
        
        if target_lang not in server_supported:
            print(f"⚠️ 번역 실패: 목표 언어 '{target_lang}'가 서버에서 지원되지 않습니다.")
            print(f"   지원 언어: {server_supported}")
            print(f"   서버 재시작 필요: --load-only ko,en,ja,{target_lang}")
            return text
        
        if source_lang != "auto" and source_lang not in server_supported:
            print(f"⚠️ 번역 실패: 소스 언어 '{source_lang}'가 서버에서 지원되지 않습니다.")
            print(f"   지원 언어: {server_supported}")
            return text
        
        try:
            response = requests.post(
                f"{self.base_url}/translate",
                json={
                    "q": text,
                    "source": source_lang,
                    "target": target_lang,
                    "format": "text"
                },
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                return result['translatedText']
            elif response.status_code == 400:
                # 400 에러 상세 정보 출력
                try:
                    error_detail = response.json()
                    print(f"⚠️ 번역 실패: 400 Bad Request")
                    print(f"   에러 내용: {error_detail}")
                except:
                    print(f"⚠️ 번역 실패: 400 Bad Request (상세 정보 없음)")
                
                # 언어 지원 확인
                if target_lang not in server_supported:
                    print(f"   → 목표 언어 '{target_lang}'가 지원되지 않습니다.")
                    print(f"   → 서버 재시작: --load-only {','.join(server_supported + [target_lang])}")
                
                return text
            else:
                print(f"⚠️ 번역 실패: HTTP {response.status_code}")
                return text
                
        except requests.exceptions.RequestException as e:
            print(f"❌ 번역 요청 오류: {str(e)}")
            return text
    
    def detect_language(self, text):
        """
        언어 감지
        
        Args:
            text: 언어를 감지할 텍스트
        
        Returns:
            감지된 언어 코드 (ko, en, ja, zh 등)
        """
        try:
            response = requests.post(
                f"{self.base_url}/detect",
                json={"q": text},
                timeout=10
            )
            
            if response.status_code == 200:
                result = response.json()
                if result and len(result) > 0:
                    detected = result[0]['language']
                    # zh-Hans, zh-Hant를 zh로 변환
                    if detected in ['zh-Hans', 'zh-Hant']:
                        return 'zh'
                    return detected
            
            # 실패 시 langdetect 사용
            return detect(text)
        except:
            # 실패 시 langdetect 사용
            return detect(text)

# LibreTranslate 클라이언트 생성
try:
    translator = LibreTranslateClient(LIBRETRANSLATE_URL)
    
    # 언어 감지 및 번역 함수
    def detect_and_translate(text, target_lang='ko'):
        """텍스트의 언어를 감지하고, 목표 언어로 번역합니다."""
        
        # 언어 감지
        detected_lang = translator.detect_language(text)
        
        # 목표 언어 정규화
        target_lang_normalized = target_lang.lower().replace('-us', '')
        
        # 언어가 목표 언어와 다른 경우 번역
        if detected_lang.lower() != target_lang_normalized:
            result = translator.translate_text(text, target_lang=target_lang_normalized, source_lang=detected_lang)
            return result, detected_lang
        
        # 언어가 목표 언어와 같은 경우 원본 텍스트 반환
        return text, detected_lang
    
    # 문서 번역 테스트 (영어 -> 한국어)
    text = "Who is the founder of Tesla?"
    
    translated_text, detected_lang = detect_and_translate(text, target_lang='ko')
    
    print(f"\nDetected language: {detected_lang}")
    print(f"Translated text: {translated_text}")
    print("✅ LibreTranslate 번역 기능이 정상적으로 작동합니다!")
    
except Exception as e:
    print(f"⚠️ LibreTranslate 클라이언트 초기화 실패: {str(e)}")
    translator = None


✅ LibreTranslate 서버 연결 성공: http://localhost:5000
   지원 언어: ['en', 'zh-Hans', 'zh-Hant', 'ja', 'ko', 'zh']

Detected language: en
Translated text: Tesla의 설립자는 누구입니까?
✅ LibreTranslate 번역 기능이 정상적으로 작동합니다!

Detected language: en
Translated text: Tesla의 설립자는 누구입니까?
✅ LibreTranslate 번역 기능이 정상적으로 작동합니다!


In [14]:
# 문서 번역 테스트 (한국어 -> 영어)

if translator is not None:
    text = "테슬라 창업자는 누구인가요?"

    translated_text, detected_lang = detect_and_translate(text, target_lang='en')

    print(f"Detected language: {detected_lang}")
    print(f"Translated text: {translated_text}")
else:
    print("⚠️ 번역 기능을 사용할 수 없습니다. LibreTranslate 서버를 실행하세요.")

Detected language: ko
Translated text: Who is Tesla Founder?


`(2) RAG 체인 성능 평가 `  

In [15]:
from langchain_core.runnables import chain, RunnablePassthrough

if translator is not None:
    # 벡터저장소 문서를 검색하는 도구 
    retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

    # 문서를 검색하여 답변을 생성하는 RAG 체인 생성
    lang_rag_chain = (
            {"context": retriever | format_docs , "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

    # 언어 감지에 기반한 RAG 실행 함수를 체인으로 변환 (@chain 데코레이터 사용)
    @chain
    def run_lang_rag_chain(query):

        # 입력 쿼리의 언어 감지
        original_lang = translator.detect_language(query)
        print(f"Original language: {original_lang}")
        
        # 한국어가 아닌 경우 번역
        if original_lang.lower() != 'ko':
            translated_query, _ = detect_and_translate(query, target_lang='ko')
        # 한국어인 경우 번역 없이 쿼리 사용
        else:
            translated_query = query

        print(f"Translated query: {translated_query}")
        
        # RAG 체인 실행
        output = lang_rag_chain.invoke(translated_query)

        print(f"Output: {output}")
        
        # 번역된 경우 다시 번역 (원래 언어로)
        if original_lang.lower() != 'ko':
            # 영어로 번역
            output = translator.translate_text(output, target_lang='en', source_lang='ko')
        
        return str(output)
    
    print("✅ 언어 감지 및 번역 기반 RAG 체인 생성 완료")
else:
    run_lang_rag_chain = None
    print("⚠️ 번역 기능을 사용할 수 없으므로 RAG 체인을 생성할 수 없습니다.")

✅ 언어 감지 및 번역 기반 RAG 체인 생성 완료


In [16]:
# 한국어 쿼리에 대한 테스트 실행

if run_lang_rag_chain is not None:
    query_ko = "테슬라 창업자는 누구인가요?"

    output = run_lang_rag_chain.invoke(query_ko)
    print(output)
else:
    print("⚠️ 번역 기능을 사용할 수 없으므로 이 셀을 스킵합니다.")

Original language: ko
Translated query: 테슬라 창업자는 누구인가요?
Output: 테슬라 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨), Ian Wright(이안 라이트)입니다.
테슬라 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨), Ian Wright(이안 라이트)입니다.
Output: 테슬라 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨), Ian Wright(이안 라이트)입니다.
테슬라 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨), Ian Wright(이안 라이트)입니다.


In [17]:
# 영어 쿼리에 대한 테스트 실행 (영어가 섞인 경우 번역 오류 발생 가능)

if run_lang_rag_chain is not None:
    query_en = "Who is the founder of Tesla?"

    output = run_lang_rag_chain.invoke(query_en)
    print(output)
else:
    print("⚠️ 번역 기능을 사용할 수 없으므로 이 셀을 스킵합니다.")

Original language: en
Translated query: Tesla의 설립자는 누구입니까?
Translated query: Tesla의 설립자는 누구입니까?
Output: Tesla의 설립자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Ian Wright(이안 라이트), Elon Musk(일론 머스크), J. B. Straubel(J. B. 스트라우벨)입니다.
Output: Tesla의 설립자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Ian Wright(이안 라이트), Elon Musk(일론 머스크), J. B. Straubel(J. B. 스트라우벨)입니다.
The founder of Tesla is Martin Eberhard, Marc Tarpenning, Ian Wright, Elon Musk, J. B. Straubel.
The founder of Tesla is Martin Eberhard, Marc Tarpenning, Ian Wright, Elon Musk, J. B. Straubel.


---

### 3. **언어 감지 및 벡터저장소 라우팅** 

- **언어별 벡터저장소**를 분리하여 한국어와 영어 문서를 독립적으로 관리함

- 각 언어에 **최적화된 저장소**를 구성하여 검색 효율성을 향상시킴

- **라우팅 시스템**을 통해 언어를 감지하고 해당 벡터저장소로 자동 연결됨

`(1) 언어별 벡터저장소 생성 `  

In [18]:
# 한국어 문서로 저장되어 있는 벡터 저장소 로드
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

db_korean = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

print(f"한국어 문서 수: {db_korean._collection.count()}")

한국어 문서 수: 39


In [19]:
# 영어 문서를 저장하는 벡터 저장소 생성
db_english = Chroma.from_documents(
    documents=english_docs, 
    embedding=embeddings_openai,
    collection_name="eng_db_openai",
    persist_directory="./chroma_db",
    )

print(f"영어 문서 수: {db_english._collection.count()}")

영어 문서 수: 12


In [20]:
# 영어 문서를 저장하는 벡터 저장소 로드
db_english = Chroma(
    embedding_function=embeddings_openai,
    collection_name="eng_db_openai",
    persist_directory="./chroma_db",
    )

print(f"영어 문서 수: {db_english._collection.count()}")

영어 문서 수: 12


`(2) RAG 체인 성능 평가 `  

In [21]:
from langdetect import detect

# 각 언어별 RAG 체인 생성 (한국어, 영어)
rag_chain_korean = create_rag_chain(db_korean)
rag_chain_english = create_rag_chain(db_english)


# 언어 감지에 기반한 RAG 실행 함수를 체인으로 변환 (@chain 데코레이터 사용)
@chain
def run_route_rag_chain(query):

    # 입력 쿼리의 언어 감지
    original_lang = detect(query)
    
    # 한국어인 경우 한국어 RAG 체인 실행 (한국어 문서 벡터 저장소 사용)
    if original_lang.upper() == 'KO':
        return rag_chain_korean.invoke(query)
    
    # 영어인 경우 영어 RAG 체인 실행 (영어 문서 벡터 저장소 사용)
    elif 'EN' in original_lang.upper():
        return rag_chain_english.invoke(query)
    
    # 한국어 또는 영어가 아닌 경우 에러 메시지 반환
    else:
        return "Unsupported language (Korean or English only)"

In [22]:
# 한국어 쿼리에 대한 테스트 실행
query_ko = "테슬라 창업자는 누구인가요?"

output = run_route_rag_chain.invoke(query_ko)
print(output)

테슬라 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨), Ian Wright(이안 라이트)입니다.


In [23]:
# 영어 쿼리에 대한 테스트 실행
query_en = "Who is the founder of Tesla?"

output = run_route_rag_chain.invoke(query_en)
print(output)

Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.


---

##  **ReAct (Reasoning and Acting)**

- **ReAct** Agent는 Reasoning과 Acting을 결합한 가장 일반적인 에이전트 형태임

- 에이전트는 **행동-관찰-추론** 단계를 순환하며 작업을 수행함
    - 행동 (act): 모델이 특정 도구(Tool)를 호출
    - 관찰 (observe): 도구의 출력(Tool Message)를 모델에 다시 전달
    - 추론 (reason): 모델이 도구 출력을 바탕으로 다음 행동을 결정 (예: 또 다른 도구를 호출하거나 직접 응답을 생성)

- **도구 호출**(act)과 **결과 분석**(observe)을 통해 다음 **행동을 결정**(reason)하는 체계적인 프로세스를 가짐


---

### 1. **도구(tool) 정의하기** 

- **ReAct 도구**는 명확한 입출력 인터페이스를 통해 정의됨

- 각 도구는 **특정 기능**을 수행하는 독립적인 컴포넌트로 구현됨

- 도구의 **입력과 출력** 형식을 명확히 정의하여 에이전트와의 상호작용을 보장함

`(1) RAG 체인 생성 (한국어, 영어 구분)`

In [24]:
# RAG 체인 생성 (메타데이터를 포함해서 답변 생성)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.chat_models import init_chat_model

# 질문 템플릿
template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "I don't know".
- For proper nouns (names of people, places, organizations, etc.), provide both Korean and English names in the format: 한글명(English Name) or English Name(한글명)
- Example: 세종대왕(King Sejong), Microsoft(마이크로소프트), New York(뉴욕)
- Use the same language as the question.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

# 프롬프트 생성
prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 생성
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 문서 포맷터 함수
def format_docs_with_metadata(docs):
    formatted_docs = []
    for doc in docs:
        content = doc.page_content
        metadata = doc.metadata
        source = metadata.get('source', '출처 없음')
        formatted_docs.append(f"내용: {content}\n출처: {source}")
    return "\n\n".join(formatted_docs)


# RAG 체인 생성 (메타데이터 포함)
def create_rag_chain_with_metadata(vectorstore, top_k=2):
    """벡터 저장소에서 문서를 검색하여 메타데이터를 포함한 답변을 생성하는 RAG 체인 생성"""

    # 벡터 저장소에서 문서를 검색
    retriever = vectorstore.as_retriever(search_kwargs={'k': top_k})
    
    chain = RunnablePassthrough.assign(
        context=lambda x: format_docs_with_metadata(retriever.invoke(x["question"]))
    ) | RunnableParallel(
        context=lambda x: x["context"],
        answer=prompt | llm | StrOutputParser()
    )
    
    return chain

# 한국어 RAG 체인 생성
rag_chain_korean = create_rag_chain_with_metadata(db_korean, top_k=4)

# 한국어 RAG 체인 실행
response = rag_chain_korean.invoke({"question": "테슬라 창업자는 누구인가요?"})

pprint(response)

{'answer': '테슬라 창업자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.',
 'context': '내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n'
            '----------------------------------\n'
            '### Roadster (2005–2009)\n'
            '\n'
            'Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞춰 적극적인 역할을 '
            '수행했습니다. 후속 자금 조달에는 Valor Equity Partners (2006)와 Sergey Brin, '
            'Larry Page, Jeff Skoll과 같은 기업가의 투자가 포함되었습니다.\n'
            '\n'
            '2007년 8월, Eberhard는 CEO에서 물러나라는 요청을 받았고, Tarpenning은 2008년 1월에 '
            "이어졌습니다. Michael Marks는 Ze'ev Drori가 인수하기 전에 임시 CEO를 역임했으며, Musk는 "
            '2008년 10월에 인수했습니다. Eberhard는 2009년 6월 Musk를 상대로 소송을 제기했지만 나중에 '
            '기각되었습니다.\n'
            '출처: data/테슬라_KR.md\n'
            '\n'
            '내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n'
            '----------------------------------\n'
            'Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Musk의 논란의 여지가 있는 발언과 '
            '관련된 소송, 정부 조사 및 비판에 직면했습니다.\n'

In [25]:
# 영어 RAG 체인 생성
rag_chain_english = create_rag_chain_with_metadata(db_english, top_k=4)

# 영어 RAG 체인 실행
response = rag_chain_english.invoke({"question": "Who is the founder of Tesla?"})

pprint(response)

{'answer': 'Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 타페닝)입니다.',
 'context': '내용: # Tesla\n'
            '\n'
            '## Company Overview\n'
            '\n'
            'Tesla, Inc. is an American electric vehicle and clean energy '
            'company. Founded in 2003, the company is headquartered in Austin, '
            'Texas, United States.\n'
            '\n'
            '## Founders\n'
            '\n'
            'Tesla was founded in 2003 by Martin Eberhard and Marc Tarpenning. '
            'Elon Musk led the Series A funding round in 2004, becoming '
            'chairman of the board, and later assumed the role of CEO.\n'
            '\n'
            '## Main Products\n'
            '\n'
            '### Electric Vehicles\n'
            '- Model S: Luxury sedan\n'
            '- Model 3: Mass-market sedan\n'
            '- Model X: Luxury SUV\n'
            '- Model Y: Mass-market SUV\n'
            '- Cybertruck: Electric pickup truck\n'
            '

`(2) RAG 체인을 Tool 객체로 변환`

In [26]:
# 한국어 RAG 도구 생성 (한국어 문서 벡터 저장소 사용)
rag_tool_korean = rag_chain_korean.as_tool(
    name="rag_korean_db",
    description="한국어 질문에 대한 리비안, 테슬라 관련 문서를 벡터 저장소에서 검색하고, 그 결과와 함께 답변을 생성합니다."
)

print(f"Tool 이름: {rag_tool_korean.name}")
print(f"Tool 설명: {rag_tool_korean.description}")
print(f"Tool 입력 파라미터: ")
pprint(rag_tool_korean.args)

Tool 이름: rag_korean_db
Tool 설명: 한국어 질문에 대한 리비안, 테슬라 관련 문서를 벡터 저장소에서 검색하고, 그 결과와 함께 답변을 생성합니다.
Tool 입력 파라미터: 
{'question': {'title': 'Question', 'type': 'string'}}


C:\Users\kaydash\AppData\Local\Temp\ipykernel_27084\2015024801.py:2: LangChainBetaWarning: This API is in beta and may change in the future.
  rag_tool_korean = rag_chain_korean.as_tool(


In [27]:
# 영어 RAG 도구 생성 (영어 문서 벡터 저장소 사용)
rag_tool_english = rag_chain_english.as_tool(
    name="rag_english_db",
    description="Retrieve and generate answers from the vector store for English questions related to Rivian and Tesla."
)

print(f"Tool 이름: {rag_tool_english.name}")
print(f"Tool 설명: {rag_tool_english.description}")
print(f"Tool 입력 파라미터: ")
pprint(rag_tool_english.args)

Tool 이름: rag_english_db
Tool 설명: Retrieve and generate answers from the vector store for English questions related to Rivian and Tesla.
Tool 입력 파라미터: 
{'question': {'title': 'Question', 'type': 'string'}}


---

### 2. **도구(tool) 호출하기** 

- **bind_tools** 메서드로 LLM에 도구들을 연결하여 사용 가능하게 함

- **도구 호출 결과**는 ToolCall 객체를 통해 체계적으로 확인할 수 있음

In [28]:
from langchain.chat_models import init_chat_model

# 도구 목록
tools = [rag_tool_korean, rag_tool_english]

# LLM 모델 
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 모델에 도구를 바인딩 (추가)
llm_with_tools = llm.bind_tools(tools=tools)

# 도구 사용하기 
query = "테슬라 창업자는 누구인가요?"
response = llm_with_tools.invoke(query)

pprint(response)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 117, 'total_tokens': 143, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOyLUl2QjULnu51oUR8E7leWnVLw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ade756c6-c448-4039-8b24-2f5a18a27344-0', tool_calls=[{'name': 'rag_korean_db', 'args': {'question': '테슬라 창업자는 누구인가요?'}, 'id': 'call_LYQUnMASboIzhYjEv8pTQHRh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 117, 'output_tokens': 26, 'total_tokens': 143, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [29]:
# ToolCall 객체 확인
response.tool_calls

[{'name': 'rag_korean_db',
  'args': {'question': '테슬라 창업자는 누구인가요?'},
  'id': 'call_LYQUnMASboIzhYjEv8pTQHRh',
  'type': 'tool_call'}]

In [30]:
# 영어 도구에 대한 질문
query_en = "Who is the founder of Tesla?"
response_en = llm_with_tools.invoke(query_en)

pprint(response_en)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 114, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOyLkPVo8Vs31GUSW8rIyn6xKaqA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ef0ecbcc-ff1d-4c79-8d3a-7bef4c0ad02b-0', tool_calls=[{'name': 'rag_english_db', 'args': {'question': 'Who is the founder of Tesla?'}, 'id': 'call_iIBbK350n6qpKplHMER2GxWf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 114, 'output_tokens': 23, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [31]:
# ToolCall 객체 확인
response_en.tool_calls

[{'name': 'rag_english_db',
  'args': {'question': 'Who is the founder of Tesla?'},
  'id': 'call_iIBbK350n6qpKplHMER2GxWf',
  'type': 'tool_call'}]

In [32]:
# 도구와 관련 없는 질문 테스트
query_test = "오늘 날씨는 어떤가요?"
response_test = llm_with_tools.invoke(query_test)

pprint(response_test)

AIMessage(content='죄송하지만, 현재 실시간 날씨 정보를 제공할 수 없습니다. 다른 도움이 필요하시면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 115, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOyMRD50438gRO9rD6uCbRVqkb4E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ef6401a0-8ae2-4203-b78f-33df2b6aaa65-0', usage_metadata={'input_tokens': 115, 'output_tokens': 25, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [33]:
# ToolCall 객체 확인
response_test.tool_calls

[]

---

### 3. **도구(tool) 실행하기** 

- **도구 호출 함수**는 AIMessage의 tool_calls를 실행하고 결과를 반환하는 헬퍼 함수로 구현

- **tool_map**을 통해 각 도구별 호출을 처리하며 invoke 메소드로 실행됨

- **최종 체인**은 llm_with_tools와 call_tools를 파이프라인으로 연결하여 구성됨

`(1) 도구 이름을 기준으로 매핑 정의`

In [34]:
# 도구 맵 생성
tool_map = {
    "rag_korean_db": rag_tool_korean,
    "rag_english_db": rag_tool_english
}

# 도구 맵을 사용하여 도구 이름을 도구 객체로 변환 (도구 이름을 키로 사용)
tool_map["rag_korean_db"].invoke({"question": "테슬라 창업자는 누구인가요?"})

{'context': "내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n----------------------------------\n### Roadster (2005–2009)\n\nElon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞춰 적극적인 역할을 수행했습니다. 후속 자금 조달에는 Valor Equity Partners (2006)와 Sergey Brin, Larry Page, Jeff Skoll과 같은 기업가의 투자가 포함되었습니다.\n\n2007년 8월, Eberhard는 CEO에서 물러나라는 요청을 받았고, Tarpenning은 2008년 1월에 이어졌습니다. Michael Marks는 Ze'ev Drori가 인수하기 전에 임시 CEO를 역임했으며, Musk는 2008년 10월에 인수했습니다. Eberhard는 2009년 6월 Musk를 상대로 소송을 제기했지만 나중에 기각되었습니다.\n출처: data/테슬라_KR.md\n\n내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n----------------------------------\nTesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Musk의 논란의 여지가 있는 발언과 관련된 소송, 정부 조사 및 비판에 직면했습니다.\n\n## 역사\n\n### 창립 (2003–2004)\n\nTesla Motors, Inc.는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 설립되었으며, 각각 CEO와 CFO를 역임했습니다. Ian Wright는 얼마 지나지 않아 합류했습니다. 2004년 2월, Elon Musk는 750만 달러의 시리즈 A 자금 조달을 주도하여 회장 겸 최대 주주가 되었습니다. J. B. Straubel은 2004년 5월 CTO로 합류했습니다. 다섯 명 모두 공동 설립자로 인정받고 있습니다.\n\n### Roadster (2005–2009)\n출처: data/ᄐ

In [35]:
# 도구 맵을 사용하여 도구 이름을 도구 객체로 변환 (영어)
tool_map["rag_english_db"].invoke({"question": "Who is the founder of Tesla?"})

{'context': '내용: # Tesla\n\n## Company Overview\n\nTesla, Inc. is an American electric vehicle and clean energy company. Founded in 2003, the company is headquartered in Austin, Texas, United States.\n\n## Founders\n\nTesla was founded in 2003 by Martin Eberhard and Marc Tarpenning. Elon Musk led the Series A funding round in 2004, becoming chairman of the board, and later assumed the role of CEO.\n\n## Main Products\n\n### Electric Vehicles\n- Model S: Luxury sedan\n- Model 3: Mass-market sedan\n- Model X: Luxury SUV\n- Model Y: Mass-market SUV\n- Cybertruck: Electric pickup truck\n\n### Energy Solutions\n- Powerwall: Home battery\n- Powerpack: Commercial battery\n- Solar Roof: Solar roofing tiles\n\n## Technological Innovation\n\nTesla is a leader in electric vehicle technology, driving innovation in battery technology, autonomous driving, and charging infrastructure.\n\n### Autonomous Driving\nTesla\'s Autopilot and Full Self-Driving (FSD) features represent some of the most advance

`(2) 도구 호출 함수 정의`

In [36]:
from langchain_core.messages import AIMessage

# 도구 호출 함수 정의
def call_tools(msg: AIMessage):
    """
    tool calling helper 함수: AIMessage에 있는 tool_calls를 실행하고 결과를 반환
    """
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


# 도구 호출 함수를 사용하여 도구 호출 실행 
print("ToolCall 객체: ")
pprint(response.tool_calls[0])
print("-"*200)

tool_calls = call_tools(response)  # 도구 호출 실행 (AIMessage 객체를 입력으로 사용)
pprint(tool_calls)

ToolCall 객체: 
{'args': {'question': '테슬라 창업자는 누구인가요?'},
 'id': 'call_LYQUnMASboIzhYjEv8pTQHRh',
 'name': 'rag_korean_db',
 'type': 'tool_call'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[{'args': {'question': '테슬라 창업자는 누구인가요?'},
  'id': 'call_LYQUnMASboIzhYjEv8pTQHRh',
  'name': 'rag_korean_db',
  'output': {'answer': '테슬라 창업자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 '
                       '타페닝)입니다.',
             'context': '내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n'
                        '----------------------------------\n'
                        '### Roadster (2005–2009)\n'
                        '\n'
                        'Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞춰 '
                        '적극적인 역할을 수행했습니다. 후속 자금 조달에는 Valor Equity Partners '
                        '(2006)와 Sergey Brin, Larry Page, Jeff Skoll

`(3) 도구 호출 및 실행 체인 정의`

In [37]:
# 도구 호출 체인 생성
search_tool_chain = llm_with_tools | call_tools

# 도구 호출 실행 (한국어 쿼리)
query = "테슬라 창업자는 누구인가요?"
search_response = search_tool_chain.invoke(query)

pprint(search_response)

[{'args': {'question': '테슬라 창업자는 누구인가요?'},
  'id': 'call_iPRuw7mU5AQzrN4BMEn4MN23',
  'name': 'rag_korean_db',
  'output': {'answer': '테슬라 창업자는 Martin Eberhard(마틴 에버하드)와 Marc Tarpenning(마크 '
                       '타페닝)입니다.',
             'context': '내용: [출처] 이 문서는 테슬라에 대한 문서입니다.\n'
                        '----------------------------------\n'
                        '### Roadster (2005–2009)\n'
                        '\n'
                        'Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞춰 '
                        '적극적인 역할을 수행했습니다. 후속 자금 조달에는 Valor Equity Partners '
                        '(2006)와 Sergey Brin, Larry Page, Jeff Skoll과 같은 기업가의 '
                        '투자가 포함되었습니다.\n'
                        '\n'
                        '2007년 8월, Eberhard는 CEO에서 물러나라는 요청을 받았고, Tarpenning은 '
                        "2008년 1월에 이어졌습니다. Michael Marks는 Ze'ev Drori가 인수하기 전에 "
                        '임시 CEO를 역임했으며, Musk는 2008년 10월에 인수했습니다. Eberhard는 '
                        '

In [38]:
# 도구 호출 실행 (영어 쿼리)
query = "Who is the founder of Tesla?"
search_response = search_tool_chain.invoke(query)

pprint(search_response)

[{'args': {'question': 'Who is the founder of Tesla?'},
  'id': 'call_Ci7VlGeVIlvJ9B8WdowV1FYS',
  'name': 'rag_english_db',
  'output': {'answer': 'Tesla의 창립자는 Martin Eberhard(마틴 에버하드)와 Marc '
                       'Tarpenning(마크 타페닝)입니다.',
             'context': '내용: # Tesla\n'
                        '\n'
                        '## Company Overview\n'
                        '\n'
                        'Tesla, Inc. is an American electric vehicle and clean '
                        'energy company. Founded in 2003, the company is '
                        'headquartered in Austin, Texas, United States.\n'
                        '\n'
                        '## Founders\n'
                        '\n'
                        'Tesla was founded in 2003 by Martin Eberhard and Marc '
                        'Tarpenning. Elon Musk led the Series A funding round '
                        'in 2004, becoming chairman of the board, and later '
                        'assumed the role o

In [39]:
# 도구 호출 실행 (한국어 쿼리) - 도구와 관련 없는 질문
query = "오늘 날씨는 어떤가요?"
search_response = search_tool_chain.invoke(query)

pprint(search_response)

[]


---

### 4. **Agent** 

- **create_agent**는 LangChain v1.0의 표준 에이전트 생성 함수

- **LLM(대규모 언어 모델)** 을 의사결정 엔진으로 사용하여 작업을 수행하는 시스템

- 에이전트의 **계획-실행-관찰** 사이클을 자동으로 관리

- 에이전트의 행동을 **모니터링**하고 결과를 반환

- LangGraph를 기반으로 구축되어 **영속성, 스트리밍, Human-in-the-loop** 등의 기능을 자동 지원


In [40]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 목록 생성 
tools = [rag_tool_korean, rag_tool_english]

# 에이전트 생성
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="당신은 사용자의 요청을 처리하는 AI Assistant입니다."
)

In [41]:
# 에이전트 실행 (한국어 쿼리)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "테슬라 창업자는 누구인가요?"}]},
)

In [42]:
# 에이전트 실행 결과 출력
pprint(response)

{'messages': [HumanMessage(content='테슬라 창업자는 누구인가요?', additional_kwargs={}, response_metadata={}, id='2839c953-13b0-403d-bba0-f8ce1504884f'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 130, 'total_tokens': 156, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOyW3Xj55Y9kijQANisSWh0rUQFs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--69f81ee5-cc92-43d6-9ba7-ce08b5c6f54e-0', tool_calls=[{'name': 'rag_korean_db', 'args': {'question': '테슬라 창업자는 누구인가요?'}, 'id': 'call_o6QVgybPjrO0XewxUy8KQyWL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 130, 'output_toke

In [43]:
# 에이전트 실행 (영어 쿼리)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Who is the founder of Tesla?"}]}
)

In [44]:
# 에이전트 실행 결과 출력
pprint(response)

{'messages': [HumanMessage(content='Who is the founder of Tesla?', additional_kwargs={}, response_metadata={}, id='f2d9ad5e-53a7-4b95-8172-5e87a671e0f1'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 127, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOyZuz0Ajl8GN0Bwy1LadtDSaPMk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--0a9c1948-4748-4c47-b361-9ca4688ce66c-0', tool_calls=[{'name': 'rag_english_db', 'args': {'question': 'Who is the founder of Tesla?'}, 'id': 'call_zLdrVU4Kp5i5NLx8pv3V2AmF', 'type': 'tool_call'}], usage_metadata={'input

In [45]:
# 에이전트 실행 (한국어 쿼리) - 도구와 관련 없는 질문
response = agent.invoke(
    {"messages": [{"role": "user", "content": "오늘 날씨는 어떤가요?"}]}
)

# 에이전트 실행 결과 출력
pprint(response)

{'messages': [HumanMessage(content='오늘 날씨는 어떤가요?', additional_kwargs={}, response_metadata={}, id='90c24fad-d5aa-475f-9654-36547281b1df'),
              AIMessage(content='죄송하지만, 현재 날씨 정보는 제공해 드릴 수 없습니다. 다른 도움이 필요하시면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 128, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CUOydTfkHU0iQsBZnHyx5L5C87xEr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--9602a885-2d70-4905-928c-95116fb5ba72-0', usage_metadata={'input_tokens': 128, 'output_tokens': 26, 'total_tokens': 154, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

---
# **[실습]**

- 언어 감지 및 번역 자동화 방식의 다국어 RAG 시스템을 구현합니다. (한국어, 영어, 중국어, 일본어 등)
- 이때, 사용자의 언어 감지 결과에 따라, 한국어와 다른 언어 간의 번역을 처리하는 도구를 별도로 구현합니다. 

- 언어 감지 결과에 따라 라우팅을 처리하고, 벡터 저장소는 한국어 DB만을 사용합니다. 

- 마지막으로, LangChain AgentExecutor 기반의 에이전트를 적용합니다. 

In [46]:
# ============================================================
# 실습: 언어 감지 및 번역 자동화 방식의 다국어 RAG 시스템
# ============================================================

# LibreTranslate 서버 확인
if translator is None:
    print("⚠️  LibreTranslate 서버가 연결되지 않았습니다!")
else:
    from langchain.tools import tool
    from langchain.agents import create_agent
    from langchain_openai import ChatOpenAI
    from typing import Literal
    import requests

    # 1. 번역 도구 정의
    @tool
    def translate_to_korean(text: str, source_lang: Literal["en", "ja", "zh", "es", "fr", "de"] = "en") -> str:
        """
        다양한 언어의 텍스트를 한국어로 번역합니다.
        
        Args:
            text: 번역할 텍스트
            source_lang: 원본 언어 코드 (en=영어, ja=일본어, zh=중국어, es=스페인어, fr=프랑스어, de=독일어)
        
        Returns:
            한국어로 번역된 텍스트
        """
        result = translator.translate_text(text, target_lang="ko", source_lang=source_lang)
        return str(result)


    @tool  
    def translate_from_korean(text: str, target_lang: Literal["en", "ja", "zh", "es", "fr", "de"] = "en") -> str:
        """
        한국어 텍스트를 다른 언어로 번역합니다.
        
        Args:
            text: 번역할 한국어 텍스트
            target_lang: 목표 언어 코드 (en=영어, ja=일본어, zh=중국어, es=스페인어, fr=프랑스어, de=독일어)
        
        Returns:
            목표 언어로 번역된 텍스트
        """
        result = translator.translate_text(text, target_lang=target_lang, source_lang="ko")
        return str(result)


    @tool
    def search_korean_documents(query: str) -> str:
        """
        한국어 문서 벡터 저장소에서 질문과 관련된 문서를 검색하고 답변을 생성합니다.
        질문은 반드시 한국어여야 합니다.
        
        Args:
            query: 한국어 질문
        
        Returns:
            검색된 문서를 기반으로 생성된 답변과 출처 정보
        """
        # RAG 체인 실행 (이미 정의된 rag_chain_korean 사용)
        response = rag_chain_korean.invoke({"question": query})
        
        # context와 answer 추출
        if isinstance(response, dict):
            context = response.get('context', '')
            answer = response.get('answer', '')
            return f"답변: {answer}\n\n참고 문서:\n{context}"
        else:
            return str(response)


    # 2. 에이전트 시스템 프롬프트 정의
    system_prompt = """당신은 다국어 질문에 답변할 수 있는 AI 어시스턴트입니다.

작동 방식:
1. 사용자의 질문 언어를 파악합니다.
2. 질문이 한국어가 아니면, translate_to_korean 도구로 한국어로 번역합니다.
3. 한국어 질문으로 search_korean_documents 도구를 사용하여 문서를 검색합니다.
4. 검색 결과를 바탕으로 답변을 생성합니다.
5. 원래 질문이 한국어가 아니었다면, translate_from_korean 도구로 원래 언어로 번역하여 답변합니다.

중요 규칙:
- search_korean_documents는 반드시 한국어 질문으로만 호출해야 합니다.
- 고유명사는 한글명(English Name) 또는 English Name(한글명) 형식으로 제공합니다.
- 문서에 없는 정보는 "모르겠습니다"라고 답변합니다.

지원 언어:
- 한국어 (ko), 영어 (en), 일본어 (ja), 중국어 (zh)
"""

    # 3. 도구 목록 및 에이전트 생성
    tools = [translate_to_korean, translate_from_korean, search_korean_documents]

    # LLM 모델 생성
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 에이전트 생성
    multilang_agent = create_agent(
        model=llm,
        tools=tools,
        system_prompt=system_prompt
    )

    print("✅ 다국어 RAG 에이전트 생성 완료!")
    print(f"📦 등록된 도구: {[tool.name for tool in tools]}")
    print(f"🌐 LibreTranslate 서버: http://localhost:5000")
    
    # supported_languages 속성 확인
    if hasattr(translator, 'supported_languages'):
        print(f"🌐 지원 언어: {translator.supported_languages}")
        # zh가 지원 언어 목록에 있는지 확인 (매핑된 값 포함)
        zh_supported = 'zh' in translator.supported_languages
    else:
        print("⚠️  지원 언어 목록을 확인할 수 없습니다. 위의 셀을 먼저 실행하세요.")
        zh_supported = False


    # 4. 테스트 실행
    print("\n" + "="*80)
    print("테스트 1: 한국어 질문")
    print("="*80)

    query_ko = "테슬라 창업자는 누구인가요?"
    print(f"질문: {query_ko}\n")

    try:
        response_ko = multilang_agent.invoke(
            {"messages": [{"role": "user", "content": query_ko}]}
        )
        print(f"답변: {response_ko['messages'][-1].content}\n")
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}\n")


    print("="*80)
    print("테스트 2: 영어 질문")
    print("="*80)

    query_en = "Who is the founder of Tesla?"
    print(f"Question: {query_en}\n")

    try:
        response_en = multilang_agent.invoke(
            {"messages": [{"role": "user", "content": query_en}]}
        )
        print(f"Answer: {response_en['messages'][-1].content}\n")
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}\n")


    print("="*80)
    print("테스트 3: 일본어 질문")
    print("="*80)

    query_ja = "テスラの創業者は誰ですか？"
    print(f"質問: {query_ja}\n")

    try:
        response_ja = multilang_agent.invoke(
            {"messages": [{"role": "user", "content": query_ja}]}
        )
        print(f"回答: {response_ja['messages'][-1].content}\n")
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}\n")


    print("="*80)
    print("테스트 4: 중국어 질문")  
    print("="*80)

    # 중국어 지원 확인
    if zh_supported:
        query_zh = "特斯拉的创始人是谁？"
        print(f"问题: {query_zh}\n")

        try:
            response_zh = multilang_agent.invoke(
                {"messages": [{"role": "user", "content": query_zh}]}
            )
            print(f"回答: {response_zh['messages'][-1].content}\n")
        except Exception as e:
            print(f"❌ 오류 발생: {str(e)}\n")
    else:
        print("⚠️ 중국어는 현재 LibreTranslate 서버에서 지원되지 않습니다.")
        if hasattr(translator, 'supported_languages'):
            # zh를 제외한 실제 서버 지원 언어 표시
            actual_langs = [lang for lang in translator.supported_languages if lang != 'zh']
            print(f"   현재 지원 언어: {actual_langs}")
        print(f"\n중국어를 사용하려면 서버를 다음과 같이 재시작하세요:")
        print(f"H:\\miniconda3\\envs\\traslate_svr\\Scripts\\libretranslate.exe --host 0.0.0.0 --port 5000 --load-only ko,en,ja,zh\n")

    print("="*80)
    print("✅ 모든 테스트 완료!")
    print("="*80)


✅ 다국어 RAG 에이전트 생성 완료!
📦 등록된 도구: ['translate_to_korean', 'translate_from_korean', 'search_korean_documents']
🌐 LibreTranslate 서버: http://localhost:5000
🌐 지원 언어: ['en', 'zh-Hans', 'zh-Hant', 'ja', 'ko', 'zh']

테스트 1: 한국어 질문
질문: 테슬라 창업자는 누구인가요?

답변: 테슬라(Tesla)의 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Ian Wright(이안 라이트), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨)입니다. 이들은 모두 테슬라의 공동 설립자로 인정받고 있습니다.

테스트 2: 영어 질문
Question: Who is the founder of Tesla?

답변: 테슬라(Tesla)의 창업자는 Martin Eberhard(마틴 에버하드), Marc Tarpenning(마크 타페닝), Ian Wright(이안 라이트), Elon Musk(일론 머스크), J. B. Straubel(제이 비 스트라우벨)입니다. 이들은 모두 테슬라의 공동 설립자로 인정받고 있습니다.

테스트 2: 영어 질문
Question: Who is the founder of Tesla?

Answer: The founders of Tesla are Martin Eberhard, Marc Tarpenning, Ian Wright, Elon Musk, and J. B. Straubel.

테스트 3: 일본어 질문
質問: テスラの創業者は誰ですか？

Answer: The founders of Tesla are Martin Eberhard, Marc Tarpenning, Ian Wright, Elon Musk, and J. B. Straubel.

테스트 3: 일본어 질문
質問: テスラの創業者は誰ですか？

回答: Tesla의 설립